In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm
from warnings import filterwarnings
import time
filterwarnings('ignore')

%matplotlib inline

PATH = '../'

In [2]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 2.65 s


In [3]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

In [4]:
training_columns = ['id', 'epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
prediction_columns = ['x','y','z','Vx', 'Vy', 'Vz']
X = train.loc[:, training_columns]
y = train.loc[:, prediction_columns]
Xtest = test.loc[:, training_columns]

In [6]:
def generateSatFeats(X, Xtest):
    X.drop(columns = ['sat_id', 'id', 'epoch'], inplace = True)
    Xtest.drop(columns = ['sat_id', 'id', 'epoch'], inplace = True)
    
    satX = pd.concat([X, X.shift(1).fillna(0), 
                      X.shift(2).fillna(0), X.shift(3).fillna(0),
                     X.shift(4).fillna(0)], axis = 1)
    satXtest = pd.concat([Xtest, Xtest.shift(1).fillna(0), Xtest.shift(2).fillna(0),
                         Xtest.shift(3).fillna(0), Xtest.shift(4).fillna(0)], axis = 1)
    return satX, satXtest

In [7]:
def train_model(X, X_test, y, folds, nTargets=6, model=None, foldWeights = [1/6, 2/6, 3/6]):
    oofSubmission = X[["id", "x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"]]
    oofSubmission.columns = ["id", "x", "y", "z", "Vx", "Vy", "Vz"]
    
    submission = X_test[["id", "x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"]]
    submission.columns = ["id", "x", "y", "z", "Vx", "Vy", "Vz"]

    oofSubmission.iloc[:, 1:] = np.zeros(oofSubmission.iloc[:, 1:].shape)
    submission.iloc[:, 1:] = np.zeros(submission.iloc[:, 1:].shape)
    
    model = model
    SMAPE = []
    for sat_id in X_test['sat_id'].unique():
        satSmape = []
        satX, satXtest = X[X['sat_id'] == sat_id], X_test[X_test['sat_id'] == sat_id]
        satX, satXtest = generateSatFeats(satX, satXtest)
        
        for fold_n, (train_index, valid_index) in enumerate(folds.split(satX)):
            foldSmape = []
            X_train, X_valid = satX.iloc[train_index], satX.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            for target in range(nTargets):
                model.fit(X_train, y_train.iloc[:, target])
                y_pred_valid = model.predict(X_valid)
                y_pred = model.predict(satXtest)
                foldSmape.append(smape(y_pred_valid, y_valid.iloc[:, target]))
                oofSubmission.loc[valid_index, oofSubmission.columns[target+1]]+=y_pred_valid * foldWeights[fold_n]
                submission.loc[satXtest.index, submission.columns[target+1]]+= y_pred* foldWeights[fold_n]
            satSmape.append(np.mean(foldSmape))
            #print(f'Fold {fold_n}. Sat {sat_id}. Score: {100*(1-np.mean(foldSmape)):.4f}')
        print(f'Sat {sat_id}. Mean Score {100*(1-np.mean(satSmape))}. Average score {100*(1-np.average(satSmape, weights = foldWeights))} ')
        print('---------------------------')
        SMAPE.append(np.mean(satSmape))
    print(f'Mean Score: {100*(1 - np.mean(SMAPE))}')
    return oofSubmission, submission

In [8]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
folds = TimeSeriesSplit(n_splits=3)
oofSubmission, submission = train_model(X,Xtest,y, folds = folds, model = LinearRegression())

Sat 1. Mean Score 63.41317304678093. Average score 60.860806895772804 
---------------------------
Sat 2. Mean Score 91.01378484524709. Average score 88.53111413714261 
---------------------------
Sat 3. Mean Score 91.0661842866026. Average score 87.21243512302509 
---------------------------
Sat 4. Mean Score 70.3223500773254. Average score 63.377139596700374 
---------------------------
Sat 6. Mean Score 91.2936707791284. Average score 89.08365779617667 
---------------------------
Sat 9. Mean Score 93.143831765495. Average score 90.13295953341594 
---------------------------
Sat 16. Mean Score 93.99791485081562. Average score 96.12501114195977 
---------------------------
Sat 20. Mean Score 92.4521038790961. Average score 94.80007397795542 
---------------------------
Sat 22. Mean Score 99.06375125735735. Average score 99.09868807825158 
---------------------------
Sat 24. Mean Score 89.99586585871059. Average score 88.83146339115932 
---------------------------
Sat 25. Mean Score 8

Sat 319. Mean Score 90.61621542654079. Average score 87.98678609768872 
---------------------------
Sat 324. Mean Score 88.81971551655603. Average score 87.38936652964787 
---------------------------
Sat 326. Mean Score 87.9364639202967. Average score 86.56001079512507 
---------------------------
Sat 332. Mean Score 89.32315814434708. Average score 88.18742674131693 
---------------------------
Sat 333. Mean Score 85.63536713144111. Average score 84.84433952303768 
---------------------------
Sat 342. Mean Score 44.642940686902186. Average score 46.52236755630314 
---------------------------
Sat 343. Mean Score 85.36477674572001. Average score 84.42751554207626 
---------------------------
Sat 344. Mean Score 56.4164902480633. Average score 52.06778862512213 
---------------------------
Sat 347. Mean Score 91.7106628577494. Average score 89.5294124818938 
---------------------------
Sat 348. Mean Score 84.67945501350674. Average score 84.33662278206707 
---------------------------
Sat

In [10]:
submission.to_csv('submission.csv', index = None)